# Natural Language Processing with Disaster Tweets
# Description
Добро пожаловать на один из наших конкурсов "Начало работы"!
Этот конкурс идеально подходит для специалистов по обработке данных, которые хотят начать работать с обработкой естественного языка. Набор данных конкурентов не слишком велик, и даже если у вас не так много персональных вычислительных мощностей, вы можете выполнять всю работу в нашей бесплатной среде Jupyter Notebooks под названием Kaggle Notebooks, которая не требует настройки.

# Dataset Description
Какие файлы мне нужны?
Вам понадобятся train.csv, test.csv и sample_submission.csv.

Какой формат данных мне следует ожидать?
Каждый образец в train и тестовом наборе содержит следующую информацию:

Текст твита
Ключевое слово из этого твита (хотя оно может быть пустым!)
Место, откуда был отправлен твит (также может быть пустым)
Что я предсказываю?
Вы предсказываете, будет ли данный твит посвящен реальной катастрофе или нет. Если да, то прогнозируем значение 1. Если нет, то прогнозируем значение 0.

Файлы
- `train.csv` - обучающий набор
- `test.csv` - тестовый набор
- `sample_submission.csv` - файл с образцом отправки в правильном формате

Столбцы
- `id` - уникальный идентификатор для каждого
- `text` - текст твита
- `location` - местоположение, из которого был отправлен твит (может быть пустым)
- `keyword` - конкретное ключевое слово из твита (может быть пустым)
- `target` - только в train.csv это означает, относится ли твит к реальной катастрофе (1) или нет (0).

In [6]:
import pandas as pd

train_path = 'nlp-getting-started/train.csv'
test_path = 'nlp-getting-started/test.csv'
ss_path = 'nlp-getting-started/sample_submission.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
ss_data = pd.read_csv(ss_path)

target_variable = 'target'

In [7]:
print(train_data.info())
train_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [10]:
X = train_data.drop(columns=[target_variable])
y = train_data[target_variable]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# categorical_features = ['location']
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.to_list()

for col in categorical_features:
    X_train[col] = X_train[col].fillna('Unknown')
    X_test[col] = X_test[col].fillna('Unknown')

from catboost import CatBoostRegressor

model = CatBoostRegressor(
    iterations=1500,
    learning_rate=0.1,
    depth=6,
    text_features=['text'],
    cat_features=categorical_features,
    verbose=100
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"RMSE: {rmse}")

0:	learn: 0.4866570	total: 171ms	remaining: 4m 15s
100:	learn: 0.4043031	total: 11.9s	remaining: 2m 44s
200:	learn: 0.3833331	total: 22.8s	remaining: 2m 27s
300:	learn: 0.3692750	total: 34.9s	remaining: 2m 19s
400:	learn: 0.3579378	total: 46.8s	remaining: 2m 8s
500:	learn: 0.3482276	total: 57.5s	remaining: 1m 54s
600:	learn: 0.3396237	total: 1m 8s	remaining: 1m 42s
700:	learn: 0.3318851	total: 1m 19s	remaining: 1m 30s
800:	learn: 0.3246693	total: 1m 29s	remaining: 1m 18s
900:	learn: 0.3175379	total: 1m 41s	remaining: 1m 7s
1000:	learn: 0.3112177	total: 1m 52s	remaining: 56.1s
1100:	learn: 0.3042084	total: 2m 5s	remaining: 45.6s
1200:	learn: 0.2985914	total: 2m 16s	remaining: 34s
1300:	learn: 0.2931235	total: 2m 27s	remaining: 22.6s
1400:	learn: 0.2878558	total: 2m 40s	remaining: 11.3s
1499:	learn: 0.2834176	total: 2m 52s	remaining: 0us
RMSE: 0.3985872166735667


In [12]:
for col in categorical_features:
    test_data[col] = test_data[col].fillna('Unknown')

test_pred = model.predict(test_data)

submission = pd.DataFrame({
    'id': test_data['id'],
    'target': test_pred
})

submission.to_csv('NLPwDT_pred_catboost.csv', index=False)

print(submission.head())
print(submission.info())

   id    target
0   0  0.581030
1   2  0.394270
2   3  0.721698
3   9  0.386842
4  11  0.727788
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      3263 non-null   int64  
 1   target  3263 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 51.1 KB
None
